In [115]:
print("안녕하세요")

안녕하세요


In [116]:
import pandas as pd
df = pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [117]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['Survived'])
y = df['Survived']

In [118]:
train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

[     PassengerId  Pclass                                           Name  \
 692          693       3                                   Lam, Mr. Ali   
 481          482       2               Frost, Mr. Anthony Wood "Archie"   
 527          528       1                             Farthing, Mr. John   
 855          856       3                     Aks, Mrs. Sam (Leah Rosen)   
 801          802       2    Collyer, Mrs. Harvey (Charlotte Annie Tate)   
 ..           ...     ...                                            ...   
 359          360       3              Mockler, Miss. Helen Mary "Ellie"   
 258          259       1                               Ward, Miss. Anna   
 736          737       3        Ford, Mrs. Edward (Margaret Ann Watson)   
 462          463       1                              Gee, Mr. Arthur H   
 507          508       1  Bradley, Mr. George ("George Arthur Brayton")   
 
         Sex   Age  SibSp  Parch      Ticket      Fare Cabin Embarked  
 692    male  

In [119]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [ ]:
X_train.shape   #규칙은 X_train만 / 훈련 점수가 당연히 높음 실제보다(정확도가)

(712, 11)

In [121]:
y_train.shape

(712,)

In [122]:
X_test.shape

(179, 11)

In [123]:
y_test.shape

(179,)

In [124]:
df.shape

(891, 12)

와 나중에 결측의 종류도 알려주신대
히히

In [125]:
for col in ["Age","Cabin","Embarked","Fare"]:
    X_train[f"{col}_isna"] = X_train[col].isna().astype(int)
    X_test[f"{col}_isna"]  = X_test[col].isna().astype(int)

In [126]:
X_train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_isna,Cabin_isna,Embarked_isna,Fare_isna
692,693,3,"Lam, Mr. Ali",male,NaN,0,0,1601,56.4958,NaN,S,1,1,0,0
481,482,2,"Frost, Mr. Anthony Wood ""Archie""",male,NaN,0,0,239854,0.0000,NaN,S,1,1,0,0
527,528,1,"Farthing, Mr. John",male,NaN,0,0,PC 17483,221.7792,C95,S,1,0,0,0
855,856,3,"Aks, Mrs. Sam (Leah Rosen)",female,18.0,0,1,392091,9.3500,NaN,S,0,1,0,0
801,802,2,"Collyer, Mrs. Harvey (Charlotte Annie Tate)",female,31.0,1,1,C.A. 31921,26.2500,NaN,S,0,1,0,0


음 Age_isna	Cabin_isna	Embarked_isna	Fare_isna 잘 생겼군

아하 얘네 결측치야? 그럼 ㅇㅋㅇㅋ 신뢰도 낮겠네? 감안하고 해볼게~

In [127]:
age_med_table = (
    X_train.groupby(["Pclass","Sex"])["Age"]
    .median()
    .reset_index()
    .rename(columns={"Age":"Age_med"})
)
age_med_table

,Pclass,Sex,Age_med
0,1,female,35.0
1,1,male,40.0
2,2,female,29.0
3,2,male,30.0
4,3,female,21.5
5,3,male,26.0


코드보고 결과가 상상이 되어야 함

In [128]:
# titanic 데이터에서 Pclass·Sex별 중앙값(Age_med)으로 결측치 채우기 임
X_train = X_train.merge(age_med_table, on=["Pclass","Sex"], how="left")
X_train["Age"] = X_train["Age"].fillna(X_train["Age_med"])
X_train = X_train.drop(columns=["Age_med"])

X_test = X_test.merge(age_med_table, on=["Pclass","Sex"], how="left")
X_test["Age"] = X_test["Age"].fillna(X_test["Age_med"])
X_test = X_test.drop(columns=["Age_med"])

train에서 중앙값을 뽑았음 -> X-test에 적용
그걸 위 코드에서 merge로 합침(결국 저 정보가 저기에 들어가니까)-> X-test에 적용(train 규칙으로)

그래서 데이터 누수가 발생한다

규칙 세움은 df에 있는 train만 , 적용은 df에 있는 train과 test 둘 다에 적용한다는 말
적용은 df에 적용한다는 말

test 데이터에 train 규칙이 들어간다(어딘가에 결측치가 있었네?! 하는 것도 앎)

test에서 규칙을 적용하지 않는 이유가 결측치를 처리하고 계산하는 규칙을 예측하게끔 그렇다면 train이랑 같은로직의 값을 넣는 이유는? -> 같은 df에서 쪼개져서 같은 형태. x_train에 있는 중앙값을 그대로 처리해도 문제가 없다(라는 전제) ㅇㅇ

### 그래서?

### 🔍 Data Leakage 방지 및 전처리 원칙
1. **Fit (규칙 수립)**: 오직 `X_train`에서만 그룹별(Pclass, Sex) 중앙값 산출
   - 이유: Test 데이터의 통계 정보(미래 지식)를 모델이 미리 학습하는 '커닝' 방지
2. **Transform (적용)**: 수립된 `train_med` 규칙을 `X_train`과 `X_test`에 동일하게 적용
   - 이유: 모델 학습 시 사용된 데이터 분포의 기준(잣대)을 평가 시에도 일관되게 유지
3. **결론**: 실전(Inference) 환경과 동일한 조건을 시뮬레이션하기 위해 Test는 오직 적용의 대상일 뿐, 규칙 생성의 주체가 될 수 없음

지금은 fit 하는 과정

X_train에 대해서만의 규칙임

그 안에서 결측치를 채움(merge에서)

# 1. fit의 역할

모델이 학습 데이터(`X_train`, `y_train`)를 보고 패턴을 학습하는 과정

쉽게 말하면, 모델이 **입력과 정답 간 관계를 “기억”**하도록 매개변수를 조정하는 단계

**예: 선형 회귀**  
목표: 

\[
y = wX + b
\]

`fit` 과정 → 손실 함수(예: MSE)를 최소화하도록 `w`, `b` 계산

---

# 2. fit 과정 단계별 흐름

* **데이터 입력 확인**  
  - X, y가 올바른 형태인지 체크  
  - 결측치, 차원 등 확인

* **모델 초기화**  
  - 파라미터를 기본값(0 혹은 랜덤)으로 초기화

* **손실 함수 정의**  
  - 모델이 얼마나 틀렸는지 평가하는 함수 설정  
  - 예: 회귀 → MSE, 분류 → Cross-Entropy

* **최적화 (Optimization)**  
  - 손실을 최소화하도록 파라미터 업데이트  
  - 경사하강법(Gradient Descent) 같은 알고리즘 사용

* **학습 완료 후 상태 저장**  
  - 최종 파라미터(`w`, `b` 등)를 모델 내부에 저장  
  - 이후 `predict` 단계에서 사용 가능

---

# 3. fit 이후

* 모델은 학습 데이터 기반 패턴을 반영한 상태  
* `predict(X_test)` → 학습한 패턴으로 예측 가능  
* `score(X_test, y_test)` → 학습 결과 평가 가능

---

💡 **핵심 요약**  

* `fit` = 모델이 데이터 보고 스스로 “배우는” 단계  
* 흐름: 입력 → 손실 계산 → 파라미터 조정 → 최종 모델 완성


늘~~~~ 기준은 train, 적용은 구분해서 train / test

In [129]:
import pandas as pd

df = pd.read_csv("train.csv")

# Sex(male/female), Embarked(S/C/Q) 원-핫 인코딩
df = pd.get_dummies(df, columns=["Sex", "Embarked"], drop_first=True)

df.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_male,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,True,False,True
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,False,False,False
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,False,False,True
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,False,False,True
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,True,False,True


- get_dummies

drop_first : 왜? 

판별이 가능하면 행 하나가 없어져도 됨

Enbarked에서 S(F) Q(F) 면 C 컬럼이 없어도? C겠구나 하잖음 ㅇㅇ
남성이 아니면 여성인 것처럼

어떤게 증가하면 같이 증가해 < 이런건 통게에서 좋지; 않이므
독립성이 좋은거 (글킨해)

결국~~~

다중공산성을 없애기 위해서이다~

get_dummies를 하면 컬럼 정렬도 하고
reindex 필수이도다 : 왜인지는 이따 TOP3에서 ㄱㄱ

`X_test = X_test.reindex(columns=X_train.columns, fill_value=0)`

이제 범주형 맞췄으니까 수치형 할 차례

자~ 스케일링 드가자

In [130]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

df = pd.read_csv("train.csv")

#결측 대치: Age 결측을 중앙값으로 채워서 Age_fill 만들기
df["Age_fill"] = df["Age"].fillna(df["Age"].median())

# Fare는 극단값이 꽤 있어서 로그 변환을 같이 쓰는 경우가 많음
df["Fare_log1p"] = np.log1p(df["Fare"])  # log(1+Fare)

# 스케일링에 사용할 수치형 열 선택
num_cols = ["Age_fill", "Fare_log1p"]

##. 사실상 이 3개만 알면 됨
####### 표준화 (Z-스코어): 평균 0, 표준편차 1
scaler_std = StandardScaler()
df[["Age_std", "Fare_std"]] = scaler_std.fit_transform(df[num_cols])

###### 0~1 정규화 (Min-Max): 최소 0, 최대 1
scaler_mm = MinMaxScaler()
df[["Age_mm", "Fare_mm"]] = scaler_mm.fit_transform(df[num_cols])

###### 로버스트 스케일: 중앙값 0, IQR(사분위 범위) 기준 (이상치에 덜 민감)
scaler_rb = RobustScaler()
df[["Age_rb", "Fare_rb"]] = scaler_rb.fit_transform(df[num_cols])

# 결과 확인
df[["Age_fill", "Fare_log1p",
    "Age_std", "Fare_std",
    "Age_mm", "Fare_mm",
    "Age_rb", "Fare_rb"]].head()

,Age_fill,Fare_log1p,Age_std,Fare_std,Age_mm,Fare_mm,Age_rb,Fare_rb
0,22.0,2.110213,-0.565736,-0.879741,0.271174,0.338125,-0.461538,-0.490934
1,38.0,4.280593,0.663861,1.361220,0.472229,0.685892,0.769231,1.206641
2,26.0,2.188856,-0.258337,-0.798540,0.321438,0.350727,-0.153846,-0.429423
3,35.0,3.990834,0.433312,1.062038,0.434531,0.639463,0.538462,0.980004
4,35.0,2.202765,0.433312,-0.784179,0.434531,0.352955,0.538462,-0.418544


## stats.zscore 안쓰고 이거?!

`scaler_std = StandardScaler()`
`df[["Age_std", "Fare_std"]] = scaler_std.fit_transform(df[num_cols])`

- 사유 : StandardScaler 에 이미 들어가있음
- 아 그러네 얘는 Scikit-learn네 R에서만 기본이지 아하
- 머신러닝 중에서 표준화할 때는 이거 많이 썼던 것 같음 ㅇㅇ (stats는 통계분석용?)
    - 데이터를 걍 넣으면 평균 표준편차 구해서 싹 하는거랑

- 이미 fit(평균과 표준편차 계산) 하고 transform(변환?적용)했으니까? 걍 이 데이터가지고 
Z = (데이터 값 - 평균) / 표준편차
- 바로 돌리면 됨 ㅋ
    - 이미 생성된 Age_std, Fare_std 자체가 그냥 zscore임

- 변환이 의도대로 됐는지 점검
    - **StandardScaler 결과**
        - 전체적으로 **평균이 0 근처**, **표준편차가 1 근처**면 정상 (딥하게 볼 필요 x)
    - **MinMaxScaler 결과**
        - 값이 **0~1 사이**면 정상(새 데이터가 학습 때 max/min을 넘으면 **1보다 크거나 0보다 작게** 나올 수 있음)
    - **RobustScaler 결과**
        - 이상치가 있는 `Fare` 같은 컬럼에서 **Standard보다 튀는 값 영향이 덜한지** 보면 됨

In [131]:
### 이상치는 판단!

# 제거할지 보정할지 뭐 어케할지 이게 참 데이터보다 막 생각하고 아 이건 측정오륜데 어쩌구블라블라 하면 됨(?)
# 캡핑이 좋음 굿입ㅂ니다 왜? 얘도 결국 정규분포를 가장 잘 이해하니까



- **처리 선택지**
    - **캡핑(윈저화)**: 극단값을 경계값으로 **덮어쓰기** → 영향 완화 + 관측 유지
    - **로그/Box-Cox/Yeo-Johnson**: 분포를 더 대칭적으로 만들어 선형 모델 적합성 ↑
    - **제거**: 측정 오류가 명백할 때만. 정보 손실과 샘플 왜곡 주의

In [132]:
#외우지마셔요 갖닸ㄲ쓰시죠 이건 공식 ㅇㅇ

import numpy as np

q1, q3 = X_train["Fare"].quantile([0.25, 0.75]) 
iqr = q3 - q1   
lo, hi = q1 - 1.5*iqr, q3 + 1.5*iqr

X_train["Fare_cap"] = X_train["Fare"].clip(lo, hi)
X_test["Fare_cap"]  = X_test["Fare"].clip(lo, hi)

X_train["Fare_log1p"] = np.log1p(X_train["Fare_cap"].clip(lower=0))
X_test["Fare_log1p"]  = np.log1p(X_test["Fare_cap"].clip(lower=0))

#X_train하되, iqr 방식으로 경계값 뽑고 train 기준으로 뽑은 그 경계값을 갖다가 클리핑(캡핑)하는 것
#여따가 로그변환까지 하면 더 안정적이다~!

이 로우와 하이의 기준이 되는 게 train임 ㅇㅇ

근데 적용은 train text 똑같이 함 (특성이 들어갈까봐)



In [133]:
X_train["Fare_log1p"]

0      4.051712
1      0.000000
2      4.199550
3      2.336987
4      3.305054
         ...   
707    2.183711
708    4.199550
709    3.566005
710    3.676301
711    3.316003
Name: Fare_log1p, Length: 712, dtype: float64

QCC : SQL 실제 테스트

## 파생변수(Feature Engineering)

> 원칙: **y(정답)를 보지 않고** X만으로 만든다.
>

In [134]:
#가족 관련

X_train["FamilySize"] = X_train["SibSp"] + X_train["Parch"] +1
X_test["FamilySize"]  = X_test["SibSp"] + X_test["Parch"] +1

X_train["IsAlone"] = (X_train["FamilySize"] ==1).astype(int)
X_test["IsAlone"]  = (X_test["FamilySize"] ==1).astype(int)

In [135]:
# SibSp 형제자매 Parch 가족 수 + 1 하면 가족 수 
# FamilySize가 1이다? 혼자 (아이고...)
X_train["FamilySize"] ==1
## -> 불리언으로 나옴

0       True
1       True
2       True
3      False
4      False
       ...  
707     True
708     True
709    False
710     True
711     True
Name: FamilySize, Length: 712, dtype: bool

In [136]:
X_train.head()

#혼자잇는걸 따로 볼 수도 있다~

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_isna,Cabin_isna,Embarked_isna,Fare_isna,Fare_cap,Fare_log1p,FamilySize,IsAlone
0,693,3,"Lam, Mr. Ali",male,26.0,0,0,1601,56.4958,NaN,S,1,1,0,0,56.4958,4.051712,1,1
1,482,2,"Frost, Mr. Anthony Wood ""Archie""",male,30.0,0,0,239854,0.0000,NaN,S,1,1,0,0,0.0000,0.000000,1,1
2,528,1,"Farthing, Mr. John",male,40.0,0,0,PC 17483,221.7792,C95,S,1,0,0,0,65.6563,4.199550,1,1
3,856,3,"Aks, Mrs. Sam (Leah Rosen)",female,18.0,0,1,392091,9.3500,NaN,S,0,1,0,0,9.3500,2.336987,2,0
4,802,2,"Collyer, Mrs. Harvey (Charlotte Annie Tate)",female,31.0,1,1,C.A. 31921,26.2500,NaN,S,0,1,0,0,26.2500,3.305054,3,0


In [137]:
#Title (Name에서 추출) + 희귀값 묶기

X_train["Title"] = X_train["Name"].str.split(",").str[1].str.split(".").str[0].str.strip()
X_test["Title"]  = X_test["Name"].str.split(",").str[1].str.split(".").str[0].str.strip()# train에서만 희귀 Title 기준 정의
title_counts = X_train["Title"].value_counts()

# 너무 희귀한 애들이 1씩 나와서 흠.. 범주로 묶고 싶다 (굳이 학습할 필요 x 의미 x 컬럼만 늘어남)
# 스킬~

rare_titles = title_counts[title_counts < 10].index # 10개 미만인 애들의 index를 뽑음

X_train["Title"] = X_train["Title"].replace(rare_titles,"Rare")
X_test["Title"]  = X_test["Title"].replace(rare_titles,"Rare")

- 누군지 보고 싶어. 그럼 바로 .index

- replace 로 바꿔버려

- 헐 홈이랑 end로 바로 할수이쑥 나 헐헐헐헐헐 

, . 뭐시기로 자꾸 하나씩 자르는 것이여

첫번째 있는 거~

In [138]:
X_train["Title"] 

#오오

0        Mr
1        Mr
2        Mr
3       Mrs
4       Mrs
       ... 
707    Miss
708    Miss
709     Mrs
710      Mr
711      Mr
Name: Title, Length: 712, dtype: object

In [139]:
X_train["Title"].value_counts()

Title
Mr        412
Miss      141
Mrs       107
Master     31
Rare       21
Name: count, dtype: int64

In [140]:
## 모델 정확도는 약~간 떨어지지만? 모델 중 100% 정확도는 없다
## 감안하고 컬럼 14 -> 5개

In [141]:
### CabinKnown

X_train["CabinKnown"] = X_train["Cabin"].notna().astype(int)
X_test["CabinKnown"]  = X_test["Cabin"].notna().astype(int)

In [142]:
## notna니까 1은 na가 아님 0은 결측임


In [143]:
## 같은 티켓값을 가진 승ㄹ객ㄱ들도
#TicketFreq (train에서만 만든 빈도표를 map)
ticket_freq = X_train["Ticket"].value_counts()
X_train["TicketFreq"] = X_train["Ticket"].map(ticket_freq).fillna(0)
X_test["TicketFreq"]  = X_test["Ticket"].map(ticket_freq).fillna(0)

- 같은 `Ticket` 값을 가진 승객이 여러 명이면
    - 그들은 **같은 예약/같은 그룹**일 가능성이 높음(가족, 동행, 단체 등)
    - 즉 같이 움직였을 가능성을 암시하는 변수
    
    `TicketFreq`는 그래서:
    
    - `1`이면 혼자 예매한 느낌
    - `2~4`면 소규모 동행
    - 더 크면 단체(드물지만)

In [144]:
#+ 비율 / 사칙연산(+, -, x, /) / 구간 혹은 범주화로 파생변수 생성 가능
# # # 구간 혹은 범주화 예시
# bins = [0, 29, 39, 49, 120]
# labels = ["20대 이하", "30대", "40대", "50대 이상"]
# df["age_group"] = pd.cut(df["age_fill"], bins=bins, labels=labels, right=True)  #pd.cut

#pd.cut

- 스케일링하면 해석은 좀 어려워짐
- 불편
- 밧 기계는 좋아함

# 결국~~? 

- **결측치 + 결측 플래그**
- **이상치(IQR 캡핑) + log1p**
- **파생변수(가족/Title+Rare/CabinKnown/Deck/TicketFreq)**
- **스케일링(StandardScaler) – train만 fit**
- **범주형 인코딩 2버전**
    - `get_dummies` + `reindex(fill_value=0)`
    - (응용) `OneHotEncoder(handle_unknown="ignore", drop="first")`

In [145]:
#공통: 데이터 로드 + SPLIT
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 1) 데이터 로드
df = pd.read_csv("train.csv")

# 2) X / y 분리
X = df.drop(columns=["Survived"])
y = df["Survived"]

# 3) SPLIT (가장 먼저!)        #데이터 나누셔
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 안전하게 copy (SettingWithCopy 방지용)
X_train = X_train.copy()
X_test = X_test.copy()

print(X_train.shape, X_test.shape)


(712, 11) (179, 11)


In [146]:
#### TRAIN에서 기준값 만들기(FIT) + 둘 다에 적용(TRANSFORM): 결측/이상치/파생변수
# - A/B 인코딩 버전(`get_dummies`/`OneHotEncoder`) 모두 여기까지는 동일하게 씀

# 1) 결측 플래그 만들기 (신호 보존)         ### 플래그 만들기
for col in ["Age", "Cabin", "Embarked", "Fare"]:
    X_train[f"{col}_isna"] = X_train[col].isna().astype(int)
    X_test[f"{col}_isna"] = X_test[col].isna().astype(int)

# 2) Embarked 결측: train 최빈값으로 채우기 (train에서만 기준값 계산)
emb_mode = X_train["Embarked"].mode()[0]
X_train["Embarked"] = X_train["Embarked"].fillna(emb_mode)
X_test["Embarked"] = X_test["Embarked"].fillna(emb_mode)

# 3) Fare 결측: train 중앙값으로 채우기 (train에서만 기준값 계산)
fare_median = X_train["Fare"].median()
X_train["Fare"] = X_train["Fare"].fillna(fare_median)
X_test["Fare"] = X_test["Fare"].fillna(fare_median)

# 4) Age 결측: (Pclass, Sex) 그룹 중앙값으로 채우기
# - 중앙값 테이블은 train에서만 만들고
# - test는 merge로 적용만 한다 (누수 방지)
age_med_table = (
    X_train.groupby(["Pclass", "Sex"])["Age"]
    .median()
    .reset_index()
    .rename(columns={"Age": "Age_med"})
)

# train에 적용
X_train = X_train.merge(age_med_table, on=["Pclass", "Sex"], how="left")
X_train["Age"] = X_train["Age"].fillna(X_train["Age_med"])
X_train = X_train.drop(columns=["Age_med"])

# test에 적용 (test에서 groupby로 새로 계산하면 안 됨!)
X_test = X_test.merge(age_med_table, on=["Pclass", "Sex"], how="left")
X_test["Age"] = X_test["Age"].fillna(X_test["Age_med"])

X_test = X_test.drop(columns=["Age_med"])

# 5) 이상치/분포 변환: Fare IQR 캡핑 + log1p
# - 캡핑 경계(lo/hi)도 train에서만 계산
q1, q3 = X_train["Fare"].quantile([0.25, 0.75])
iqr = q3 - q1
lo = q1 - 1.5 * iqr
hi = q3 + 1.5 * iqr

X_train["Fare_cap"] = X_train["Fare"].clip(lo, hi)      #clip 클리핑 함~
X_test["Fare_cap"] = X_test["Fare"].clip(lo, hi)

X_train["Fare_log1p"] = np.log1p(X_train["Fare_cap"].clip(lower=0))
X_test["Fare_log1p"] = np.log1p(X_test["Fare_cap"].clip(lower=0))

# 6) 파생변수(Feature Engineering)
# 6-1) 가족 변수
X_train["FamilySize"] = X_train["SibSp"] + X_train["Parch"] + 1
X_test["FamilySize"]  = X_test["SibSp"]  + X_test["Parch"]  + 1

X_train["IsAlone"] = (X_train["FamilySize"] == 1).astype(int)
X_test["IsAlone"]  = (X_test["FamilySize"] == 1).astype(int)

# 6-2) Title 추출
X_train["Title"] = X_train["Name"].str.split(",").str[1].str.split(".").str[0].str.strip()
X_test["Title"]  = X_test["Name"].str.split(",").str[1].str.split(".").str[0].str.strip()

# 6-3) CabinKnown
X_train["CabinKnown"] = X_train["Cabin"].notna().astype(int)
X_test["CabinKnown"]  = X_test["Cabin"].notna().astype(int)

# 6-4) Title 희귀값 Rare로 묶기 (기준은 train에서만!)
title_counts = X_train["Title"].value_counts()
rare_titles = title_counts[title_counts < 10].index

X_train["Title"] = X_train["Title"].replace(rare_titles, "Rare")
X_test["Title"]  = X_test["Title"].replace(rare_titles, "Rare")

# 6-5) TicketFreq (train에서만 빈도표 만들기)
ticket_freq = X_train["Ticket"].value_counts()
X_train["TicketFreq"] = X_train["Ticket"].map(ticket_freq).fillna(0)
X_test["TicketFreq"]  = X_test["Ticket"].map(ticket_freq).fillna(0)

# 7) 모델 입력에서 빼도 되는 원본 텍스트 컬럼 제거      ***
# - Name/Ticket/Cabin은 기본 모델에서 바로 쓰기 어려움
drop_cols = ["Name", "Ticket", "Cabin", "Fare", "Fare_cap"]  # Fare는 Fare_log1p로 대체
X_train = X_train.drop(columns=drop_cols)
X_test = X_test.drop(columns=drop_cols)

print("after feature engineering:", X_train.shape, X_test.shape)



after feature engineering: (712, 17) (179, 17)


In [154]:
#### 범주형 인코딩 A: **pandas get_dummies 버전**

# - `get_dummies(drop_first=True)`는 k개 범주를 k-1 더미로 만들어주는 옵션(drop_first)이 있고 
# `reindex(..., fill_value=0)`로 **train 컬럼 기준으로 test 컬럼을 맞춥니다**.

# 1) 범주형 컬럼 지정 (원핫할 것들)
cat_cols = ["Sex", "Embarked", "Title", "Pclass"]

# 2) get_dummies (train/test 각각)      ### get_dummies
X_train_dum = pd.get_dummies(X_train, columns=cat_cols, drop_first=True)
X_test_dum  = pd.get_dummies(X_test,  columns=cat_cols, drop_first=True)

# 3) train 기준으로 test 컬럼 맞추기 (매우 중요!)
X_test_dum = X_test_dum.reindex(columns=X_train_dum.columns, fill_value=0)

print("after dummies:", X_train_dum.shape, X_test_dum.shape)

# 4) 스케일링(수치형만) - StandardScaler는 train에서만 fit
scale_cols = ["Age", "Fare_log1p", "SibSp", "Parch", "FamilySize", "TicketFreq", "FarePerPerson"]   #수치형데이터
scale_cols = [c for c in scale_cols if c in X_train_dum.columns]  # 안전장치

scaler = StandardScaler()
X_train_dum[scale_cols] = scaler.fit_transform(X_train_dum[scale_cols])  # train fit+transform
X_test_dum[scale_cols]  = scaler.transform(X_test_dum[scale_cols])       # test transform only

####------------ X 피처를 만들었다(입력피처를 만드는 게 목적이었고(오늘) 끝! 해냄)

# 5) 모델 학습(예시: 로지스틱 회귀)
model = LogisticRegression(max_iter=1000)
model.fit(X_train_dum, y_train)

# 6) 평가
pred = model.predict(X_test_dum)
print("Accuracy:", accuracy_score(y_test, pred))

after dummies: (712, 22) (179, 22)
Accuracy: 0.8100558659217877


d:\sonamu\campstudy_Python\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [153]:
### 안전장치 (위에있던거)

scale_cols = ["Age", "Fare_log1p", "SibSp", "Parch", "FamilySize", "TicketFreq", "FarePerPerson"]   #수치형데이터
scale_cols = [c for c in scale_cols if c in X_train_dum.columns]  # 안전장치 (리스트 컴프리헨션)

# c for c in scale_cols if c in X_train_dum.columns
# 뭐고이거
# 오? c for c네 in scale_cols(수치형 변수)에 있는거 하나하나 꺼내라 if 만약에 in (있으면) c 안에 추가하라(리스트 안에) 없으면? 넣지마넣지마 ~ 있는 것만 넣어
    # 만약에 없으면? (수치형 변수가?)
    # 없으면 넣지 마/ 있는 애들만 넣어
#혹시나~~를 위해서 (소실됐거나/수치형이 아니거나/) <- 걍 info 찍으면 되는 거아님? (맞음) 근데 혹시나~~

In [148]:
#### 범주형 인코딩 B: **sklearn OneHotEncoder 버전**

# - `OneHotEncoder(handle_unknown="ignore")`는 **test에서 처음 보는 범주가 나와도**  해당 원핫이 0으로 처리됩니다.

import numpy as np
from sklearn.preprocessing import OneHotEncoder

# 1) 범주형/수치형 컬럼 분리
cat_cols = ["Sex", "Embarked", "Title", "Pclass"]
num_cols = ["Age", "Fare_log1p", "SibSp", "Parch", "FamilySize", "TicketFreq"]

# 2) OneHotEncoder 준비 (버전 차이 대응)
try:
    ohe = OneHotEncoder(handle_unknown="ignore", drop="first", sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", drop="first", sparse=False)

X_train_cat = ohe.fit_transform(X_train[cat_cols])  # train: fit_transform
X_test_cat  = ohe.transform(X_test[cat_cols])       # test : transform only

# 3) 수치형 스케일링 (train에서만 fit)
scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train[num_cols])
X_test_num  = scaler.transform(X_test[num_cols])

# 4) 나머지(결측 플래그, IsAlone, CabinKnown 같은 이진/정수 컬럼) 붙이기
# -> 여기서는 cat_cols, num_cols 빼고 남은 것을 통째로 붙임
rest_cols = [c for c in X_train.columns if c not in cat_cols + num_cols]
X_train_rest = X_train[rest_cols].to_numpy(dtype=float)
X_test_rest  = X_test[rest_cols].to_numpy(dtype=float)

# 5) 최종 X 합치기
X_train_final = np.hstack([X_train_num, X_train_rest, X_train_cat])
X_test_final  = np.hstack([X_test_num,  X_test_rest,  X_test_cat])

# 6) 모델 학습/평가
model = LogisticRegression(max_iter=1000)
model.fit(X_train_final, y_train)

pred = model.predict(X_test_final)
print("Accuracy:", accuracy_score(y_test, pred))

d:\sonamu\campstudy_Python\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:261: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)


Accuracy: 0.8044692737430168


d:\sonamu\campstudy_Python\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
